<a href="https://colab.research.google.com/github/kimyuji/RecSys_DSAIL/blob/main/%5BWeek4%5DWide%26Deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Wide&Deep 노션 정리
https://www.notion.so/Wide-Deep-Wide-Deep-Learning-for-Recommender-Systems-ac1869aa78e040a3a39d7aff6ce1d4f4



In [ ]:
!pip install pytorch-widedeep

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

## 1. Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/[인턴]2020겨울학기_DSAIL/ml-100k/"
%cd $path

/content/drive/MyDrive/Colab Notebooks/[인턴]2020겨울학기_DSAIL/ml-100k


### Rating

In [ ]:
df = pd.read_csv(os.path.join(path,'u.data'), sep='\t', names=['user_id', 'item_id', 'rating', 'time_stamp'])
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1234, shuffle=True)

In [ ]:
df.dtypes

user_id       int64
item_id       int64
rating        int64
time_stamp    int64
dtype: object

In [ ]:
df

,user_id,item_id,rating,time_stamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


### User feature

In [ ]:
user_df = pd.read_csv(os.path.join(path + 'u.user'), sep='|', 
            encoding='utf-8', names = ['user_id','age','gender','occupation','zipcode'])
user_df['user_id'].astype(int)
user_df = user_df.set_index('user_id')
print(user_df.dtypes)
user_df

age            int64
gender        object
occupation    object
zipcode       object
dtype: object


,age,gender,occupation,zipcode
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


### Item feature

In [ ]:
item_df = pd.DataFrame(columns=['item_id', 'movie_name', 'release', 'video_release', 'url','unknown', 'Action','Adventure','Animation',
              'Children','Comedy','Crime','Documentary','Drama','Fantasy',
              'Film-Noir','Horror','Musical','Mystery','Romance', 'Sci-Fi',
              'Thriller', 'War','Western'])
r = open(path+'u.item', mode='rt', encoding='latin-1')
for line in r:
  item_df.loc[len(item_df)] = line.replace("\n","").split("|")
r.close()
item_df.drop(266, inplace=True)
item_df['release'] = pd.to_datetime(item_df['release'], format='%d-%b-%Y')
#item_df['item_id'].astype(int)
#item_df = item_df.set_index('item_id')
print(item_df.dtypes)
display(item_df)

item_id                  object
movie_name               object
release          datetime64[ns]
video_release            object
url                      object
unknown                  object
Action                   object
Adventure                object
Animation                object
Children                 object
Comedy                   object
Crime                    object
Documentary              object
Drama                    object
Fantasy                  object
Film-Noir                object
Horror                   object
Musical                  object
Mystery                  object
Romance                  object
Sci-Fi                   object
Thriller                 object
War                      object
Western                  object
dtype: object


,item_id,movie_name,release,video_release,url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1995-01-01,,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1995-01-01,,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),1998-02-06,,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),1998-02-06,,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),1998-01-01,,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),1994-01-01,,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### 1. Genre One-hot vector

In [ ]:
dummy_genres_df = pd.concat([item_df[['item_id']], item_df.iloc[:,6:24]], axis=1)
dummy_genres_df[:].astype(int)
dummy_genres_df = dummy_genres_df.apply(pd.to_numeric)
print(dummy_genres_df.dtypes)
dummy_genres_df

item_id        int64
Action         int64
Adventure      int64
Animation      int64
Children       int64
Comedy         int64
Crime          int64
Documentary    int64
Drama          int64
Fantasy        int64
Film-Noir      int64
Horror         int64
Musical        int64
Mystery        int64
Romance        int64
Sci-Fi         int64
Thriller       int64
War            int64
Western        int64
dtype: object


,item_id,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. User Features One-hot vector

In [ ]:
# genre one-hot vector
dummy_gender = pd.get_dummies(user_df.gender)
# age
age = user_df.age
# occupation
dummy_occupation = pd.get_dummies(user_df.occupation)
user_features = pd.concat([dummy_gender, age, dummy_occupation], axis=1)

### 3. Normalized Time delta
가장 먼저 개봉한 영화와 몇달 차이로 개봉했는지

In [ ]:
start_date = item_df['release'].min()
# time_delta
item_df['month_delta'] = ((item_df['release'] - start_date)/np.timedelta64(1, 'M')).astype(int)
# normalize
time_df = (item_df['month_delta']-item_df['month_delta'].min())/(item_df['month_delta'].max()-item_df['month_delta'].min())
time_df = pd.concat([item_df[['item_id']].astype(int), time_df], axis=1)

In [ ]:
train_df = pd.merge(df, user_features, left_on='user_id', right_on='user_id', how='left')
train_df = pd.merge(train_df, dummy_genres_df, left_on='item_id', right_on='item_id', how='left')
train_df = pd.merge(train_df, time_df, left_on='item_id', right_on='item_id', how='left')
train_df

,user_id,item_id,rating,time_stamp,F,M,age,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,homemaker,lawyer,librarian,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,month_delta
0,196,242,3,881250949,0,1,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.977199
1,186,302,3,891717742,1,0,39,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.977199
2,22,377,1,878887116,0,1,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.938111
3,244,51,2,880606923,0,1,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.938111
4,166,346,1,886397596,0,1,47,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.977199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,880175444,0,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.972856
99996,716,204,5,879795543,1,0,36,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.820847
99997,276,1090,1,874795795,0,1,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.925081
99998,13,225,2,882399156,0,1,47,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.975027


In [ ]:
train_df.columns[4:]

Index(['F', 'M', 'age', 'administrator', 'artist', 'doctor', 'educator',
       'engineer', 'entertainment', 'executive', 'healthcare', 'homemaker',
       'lawyer', 'librarian', 'marketing', 'none', 'other', 'programmer',
       'retired', 'salesman', 'scientist', 'student', 'technician', 'writer',
       'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'month_delta'],
      dtype='object')

In [ ]:
wide_cols = list(train_df.columns[4:6]) + list(train_df.columns[28:47])
wide_cols

['F',
 'M',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'month_delta']

### wide component에서의 cross component

In [ ]:
print(len(wide_cols))
print(wide_cols)

3
['F', 'M', 'age']


In [ ]:
import itertools
from itertools import product  
cross_cols = []
cross_cols.append(list(train_df.columns[4:6]))
cross_cols.append(list(train_df.columns[28:47]))
cross_cols = list(product(*cross_cols))
print(cross_cols)

[('F', 'Action'), ('F', 'Adventure'), ('F', 'Animation'), ('F', 'Children'), ('F', 'Comedy'), ('F', 'Crime'), ('F', 'Documentary'), ('F', 'Drama'), ('F', 'Fantasy'), ('F', 'Film-Noir'), ('F', 'Horror'), ('F', 'Musical'), ('F', 'Mystery'), ('F', 'Romance'), ('F', 'Sci-Fi'), ('F', 'Thriller'), ('F', 'War'), ('F', 'Western'), ('F', 'month_delta'), ('M', 'Action'), ('M', 'Adventure'), ('M', 'Animation'), ('M', 'Children'), ('M', 'Comedy'), ('M', 'Crime'), ('M', 'Documentary'), ('M', 'Drama'), ('M', 'Fantasy'), ('M', 'Film-Noir'), ('M', 'Horror'), ('M', 'Musical'), ('M', 'Mystery'), ('M', 'Romance'), ('M', 'Sci-Fi'), ('M', 'Thriller'), ('M', 'War'), ('M', 'Western'), ('M', 'month_delta')]


In [ ]:
# # cross column 만들기
# import itertools
# from itertools import product  
# unique_combinations = list(list(zip(wide_cols, element)) 
#                            for element in product(wide_cols, repeat = len(wide_cols))) 

# print(unique_combinations)
# # 겹치는 cross column 삭제
# cross_cols = [item for sublist in unique_combinations for item in sublist]
# cross_cols = [x for x in cross_cols if x[0] != x[1]]
# cross_cols = list(set(cross_cols))
# print(cross_cols)

[[('F', 'F'), ('M', 'F'), ('age', 'F')], [('F', 'F'), ('M', 'F'), ('age', 'M')], [('F', 'F'), ('M', 'F'), ('age', 'age')], [('F', 'F'), ('M', 'M'), ('age', 'F')], [('F', 'F'), ('M', 'M'), ('age', 'M')], [('F', 'F'), ('M', 'M'), ('age', 'age')], [('F', 'F'), ('M', 'age'), ('age', 'F')], [('F', 'F'), ('M', 'age'), ('age', 'M')], [('F', 'F'), ('M', 'age'), ('age', 'age')], [('F', 'M'), ('M', 'F'), ('age', 'F')], [('F', 'M'), ('M', 'F'), ('age', 'M')], [('F', 'M'), ('M', 'F'), ('age', 'age')], [('F', 'M'), ('M', 'M'), ('age', 'F')], [('F', 'M'), ('M', 'M'), ('age', 'M')], [('F', 'M'), ('M', 'M'), ('age', 'age')], [('F', 'M'), ('M', 'age'), ('age', 'F')], [('F', 'M'), ('M', 'age'), ('age', 'M')], [('F', 'M'), ('M', 'age'), ('age', 'age')], [('F', 'age'), ('M', 'F'), ('age', 'F')], [('F', 'age'), ('M', 'F'), ('age', 'M')], [('F', 'age'), ('M', 'F'), ('age', 'age')], [('F', 'age'), ('M', 'M'), ('age', 'F')], [('F', 'age'), ('M', 'M'), ('age', 'M')], [('F', 'age'), ('M', 'M'), ('age', 'age')],

### deep의 columns

In [ ]:
# embed_cols = [('genre', 16),('grade', 16)]
# 16 = embedding dimension
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['age', 'month_delta']

print(embed_cols)
print(continuous_cols)

[('F', 16), ('M', 16)]
['age', 'month_delta']


In [ ]:
target = train_df['rating'].values

# Wide&Deep

In [ ]:
from pytorch_widedeep.preprocessing import WidePreprocessor, DensePreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import Accuracy

## Wide Columns

In [ ]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = preprocess_wide.fit_transform(train_df)
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [ ]:
X_wide

In [ ]:
wide

Wide(
  (wide_linear): Embedding(31, 1, padding_idx=0)
)

## Deep Columns

In [ ]:
preprocess_deep = DensePreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols)
X_deep = preprocess_deep.fit_transform(train_df)
deepdense = DeepDense(
    hidden_layers=[64, 32],
    deep_column_idx=preprocess_deep.deep_column_idx,
    embed_input=preprocess_deep.embeddings_input,
    continuous_cols=continuous_cols,
)

## Build and Train

In [ ]:
model = WideDeep(wide=wide, deepdense=deepdense) # 동시에 넣어주기
model.compile(method="binary", metrics=[Accuracy])
model.fit(
    X_wide=X_wide,
    X_deep=X_deep,
    target=target,
    n_epochs=5,
    batch_size=256,
    val_split=0.1,
)